In [1]:
import os
import pandas as pd

import scanpy as sc
import random, torch
import warnings
warnings.filterwarnings("ignore")
import SpaGCN as spg
#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#inatll opencv in python
#!pip3 install opencv-python
import cv2
from anndata import AnnData
from sklearn.metrics import accuracy_score
from sklearn.metrics import adjusted_rand_score
import numpy as np
from sklearn.metrics import confusion_matrix
from scipy.optimize import linear_sum_assignment

In [2]:
for slicename in range(25, 30):  # 25  29
    slicename = str(slicename)
    adata = sc.read_h5ad(fr"C:\E\JSU\BIO\file\STrafer\params\merfish\{slicename}.h5ad")
    label_mapping = {'MPA': 1, 'MPN': 2, 'BST': 3, 'fx': 4, "PVH":5,"PVT":6,"V3":7, 'PV': 0}
    labels = adata.obs['ground_truth'].map(label_mapping)
    adata.var_names_make_unique()
    # pre-process
    adata.layers['count'] = adata.X
    sc.pp.filter_genes(adata, min_cells=50)
    sc.pp.filter_genes(adata, min_counts=10)
    sc.pp.normalize_total(adata, target_sum=1e6)
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", layer='count', n_top_genes=150)
    adata = adata[:, adata.var['highly_variable'] == True]
    sc.pp.scale(adata)    
    x_array = adata.obsm['spatial'][:, 0]
    y_array = adata.obsm['spatial'][:, 1]    
    s=1
    b=49
    adj=spg.calculate_adj_matrix(x=x_array,y=y_array, x_pixel=x_array, y_pixel=y_array, beta=b, alpha=s, histology=False)
    
    folder_path = fr'C:\E\JSU\BIO\file\STrafer\params\merfish\SPAGCN\{slicename}'
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, 'adj.csv')
    np.savetxt(file_path, adj, delimiter=',')
    
    adj=np.loadtxt(file_path, delimiter=',')
    p=0.5
    #Find the l value given p
    l=spg.search_l(p, adj, start=0.01, end=1000, tol=0.01, max_run=100)    
    n_clusters=8
    #Set seed
    r_seed=t_seed=n_seed=100
    #Search for suitable resolution
    res=spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed, t_seed=t_seed, n_seed=n_seed)
    
    clf=spg.SpaGCN()
    clf.set_l(l)
    #Set seed
    random.seed(r_seed)
    torch.manual_seed(t_seed)
    np.random.seed(n_seed)
    #Run
    clf.train(adata,adj,init_spa=True,init="louvain",res=res, tol=5e-3, lr=0.05, max_epochs=200)
    y_pred, prob=clf.predict()

Calculateing adj matrix using xy only...
Run 1: l [0.01, 1000], p [0.0, 3620.6197702048403]
Run 2: l [0.01, 500.005], p [0.0, 1739.4111349480254]
Run 3: l [0.01, 250.0075], p [0.0, 571.4368495576911]
Run 4: l [0.01, 125.00874999999999], p [0.0, 159.58733009197778]
Run 5: l [0.01, 62.509375], p [0.0, 41.85332553130277]
Run 6: l [0.01, 31.2596875], p [0.0, 10.534401514335222]
Run 7: l [0.01, 15.63484375], p [0.0, 2.424396455767176]
Run 8: l [7.822421875, 15.63484375], p [0.4425786968261767, 2.424396455767176]
Run 9: l [7.822421875, 11.728632812499999], p [0.4425786968261767, 1.2414307603567876]
Run 10: l [7.822421875, 9.77552734375], p [0.4425786968261767, 0.7917804443247025]
Run 11: l [7.822421875, 8.798974609375], p [0.4425786968261767, 0.6044623855519853]
Run 12: l [7.822421875, 8.310698242187499], p [0.4425786968261767, 0.5203384404306686]
Run 13: l [8.06656005859375, 8.310698242187499], p [0.4806640586871078, 0.5203384404306686]
recommended l =  8.188629150390625
Start at res =  0.7

In [3]:
    conf_mat = confusion_matrix(labels, y_pred, labels=np.arange(8))
    row_ind, col_ind = linear_sum_assignment(-conf_mat)
    mapping = {pred_label: true_label for true_label, pred_label in zip(row_ind, col_ind)}
    y_pred = np.array([mapping[p] for p in y_pred])    
    pred_labels_list = pd.DataFrame({
        'spot': list(range(1, len(y_pred) + 1)),
        'pred': y_pred
    })
    file_path_pred = os.path.join(folder_path, 'pred_labels.csv')
    pred_labels_list.to_csv(file_path_pred, index=False) 

In [4]:
    ARI_s = adjusted_rand_score(y_pred, labels)
    acc_s = accuracy_score(y_pred, labels)
    print("ARI_s:", ARI_s)
    print("acc_s", acc_s)

ARI_s: 0.216688343380537
acc_s 0.4783156379341371
